In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import pandas as pd
from pathlib import Path
import time, copy


In [2]:
# Load data

X_train = pd.read_csv(Path('cleaned_data','X_train.csv'), index_col=0)
X_val = pd.read_csv(Path('cleaned_data','X_val.csv'), index_col=0)
X_test = pd.read_csv(Path('cleaned_data','X_test.csv'), index_col=0)

y_train = pd.read_csv(Path('cleaned_data','y_train.csv'), index_col=0)
y_val = pd.read_csv(Path('cleaned_data','y_val.csv'), index_col=0)
y_test = pd.read_csv(Path('cleaned_data','y_test.csv'), index_col=0)

In [3]:
# Define device

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [4]:
# Define custom dataset

class MimicIvDataset(Dataset):
    """MIMIC IV dataset."""

    def __init__(self, csv_file_X, csv_file_y):
        """
        Arguments:
            csv_file (string): Path to the csv file with annotations.
        """
        self.mimic_df_X = pd.read_csv(Path(csv_file_X), index_col=0)
        self.mimic_df_y = pd.read_csv(Path(csv_file_y), index_col=0)

    def __len__(self):
        return len(self.mimic_df_X)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()
            
        inputs = torch.tensor(self.mimic_df_X.iloc[idx], dtype=torch.float64)
        labels = torch.tensor(self.mimic_df_y.iloc[idx], dtype=torch.float64)

        return inputs, labels

In [5]:
# Load datasets

train_dataset = MimicIvDataset(csv_file_X="cleaned_data/X_train.csv", csv_file_y="cleaned_data/y_train.csv")
val_dataset = MimicIvDataset(csv_file_X="cleaned_data/X_val.csv", csv_file_y="cleaned_data/y_val.csv")
test_dataset = MimicIvDataset(csv_file_X="cleaned_data/X_test.csv", csv_file_y="cleaned_data/y_test.csv")

In [6]:
# Create dataloaders

batch_size = 100

dataloaders = {'train': DataLoader(train_dataset, batch_size=batch_size, shuffle=True),
               'val': DataLoader(val_dataset, batch_size=batch_size, shuffle=True),
               'test': DataLoader(test_dataset, batch_size=batch_size, shuffle=True)}

dataset_sizes = {'train': len(train_dataset),
                 'val': len(val_dataset),
                 'test': len(test_dataset)}
print(f'dataset_sizes = {dataset_sizes}')

dataset_sizes = {'train': 344320, 'val': 38258, 'test': 42509}


In [7]:
import torch
import torch.nn as nn

# Define model parameters
input_size = 67
hidden_size1 = 128
hidden_size2 = 128
hidden_size3 = 128
num_classes = 1
dropout_rate = 0.5  # Adjust the dropout rate for regularization

# External training parameters
learning_rate = 0.001
num_epochs = 10

# Revised model with flexible activation and data types
class MimicAdmissionClassifier(nn.Module):
    def __init__(self, input_size, hidden_size1, hidden_size2, hidden_size3, num_classes, dropout_rate, activation_fn=nn.ReLU, dtype=torch.float32):
        """
        A customizable multi-layer perceptron (MLP) model with dropout for classification tasks.

        Args:
            input_size (int): Size of the input features.
            hidden_size1 (int): Number of units in the first hidden layer.
            hidden_size2 (int): Number of units in the second hidden layer.
            hidden_size3 (int): Number of units in the third hidden layer.
            num_classes (int): Number of output classes (1 for binary classification).
            dropout_rate (float): Dropout rate to use between layers (0 for no dropout).
            activation_fn (torch.nn.Module): Activation function (default is ReLU).
            dtype (torch.dtype): Data type for the model (default is torch.float32).
        """
        super(MimicAdmissionClassifier, self).__init__()
        self.dropout = nn.Dropout(dropout_rate)  # Dropout layer to prevent overfitting
        
        # Define the sequence of layers with dynamic activation function
        self.layers = nn.Sequential(
            nn.Linear(input_size, hidden_size1, dtype=dtype),
            activation_fn(),
            self.dropout,
            nn.Linear(hidden_size1, hidden_size2, dtype=dtype),
            activation_fn(),
            self.dropout,
            nn.Linear(hidden_size2, hidden_size3, dtype=dtype),
            activation_fn(),
            self.dropout,
            nn.Linear(hidden_size3, num_classes, dtype=dtype),
            nn.Sigmoid()  # For binary classification
        )

    def forward(self, x):
        """
        Forward pass of the network.

        Args:
            x (Tensor): Input data tensor of shape (batch_size, input_size).
        
        Returns:
            Tensor: Output prediction tensor of shape (batch_size, num_classes).
        """
        return self.layers(x)

# Example instantiation of the model
mimic_admission_classifier = MimicAdmissionClassifier(
    input_size=input_size,
    hidden_size1=hidden_size1,
    hidden_size2=hidden_size2,
    hidden_size3=hidden_size3,
    num_classes=num_classes,
    dropout_rate=dropout_rate,
    activation_fn=nn.ReLU,  # Can be changed to other activations like nn.LeakyReLU
    dtype=torch.float64  # Data type can be changed if needed
).to(device)

# Print model summary (optional)
print(mimic_admission_classifier)


MimicAdmissionClassifier(
  (dropout): Dropout(p=0.5, inplace=False)
  (layers): Sequential(
    (0): Linear(in_features=67, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5, inplace=False)
    (3): Linear(in_features=128, out_features=128, bias=True)
    (4): ReLU()
    (5): Dropout(p=0.5, inplace=False)
    (6): Linear(in_features=128, out_features=128, bias=True)
    (7): ReLU()
    (8): Dropout(p=0.5, inplace=False)
    (9): Linear(in_features=128, out_features=1, bias=True)
    (10): Sigmoid()
  )
)


In [10]:
# From https://pytorch.org/tutorials/beginner/transfer_learning_tutorial.html

def train_model(model, dataloaders, dataset_sizes, criterion, optimizer, scheduler, num_epochs=25):
    since = time.time()

    best_model_wts = copy.deepcopy(model.state_dict()) # keep the best weights stored separately
    best_acc = 0.0
    best_epoch = 0

    # Each epoch has a training, validation, and test phase
    phases = ['train', 'val']
    
    # Keep track of how loss and accuracy evolves during training
    training_curves = {}
    for phase in phases:
        training_curves[phase+'_loss'] = []
        training_curves[phase+'_acc'] = []
    
    for epoch in range(num_epochs):
        print(f'\nEpoch {epoch+1}/{num_epochs}')
        print('-' * 10)

        for phase in phases:
            if phase == 'train':
                model.train()  # Set model to training mode
            else:
                model.eval()   # Set model to evaluate mode

            running_loss = 0.0
            running_corrects = 0

            # Iterate over data.
            for inputs, labels in dataloaders[phase]:
                # This ensures all of our datapoints are flattened
                # before feeding them to our model
                inputs = inputs.view(inputs.shape[0],-1)
                inputs = inputs.to(device)
                labels = labels.to(device)

                # zero the parameter gradients
                optimizer.zero_grad()

                # forward
                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    _, predictions = torch.max(outputs, 1)
                    loss = criterion(outputs, labels)

                    # backward + update weights only if in training phase
                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                # statistics
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(predictions == labels.data)
 
            if phase == 'train':
                scheduler.step()

            epoch_loss = running_loss / dataset_sizes[phase]
            epoch_acc = running_corrects.double() / dataset_sizes[phase]
            training_curves[phase+'_loss'].append(epoch_loss)
            training_curves[phase+'_acc'].append(epoch_acc)

            print(f'{phase:5} Loss: {epoch_loss:.4f} Acc: {epoch_acc:.4f}')

            # deep copy the model if it's the best accuracy (based on validation)
            if phase == 'val' and epoch_acc > best_acc:
                best_epoch = epoch
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())

    time_elapsed = time.time() - since
    print(f'\nTraining complete in {time_elapsed // 60:.0f}m {time_elapsed % 60:.0f}s')
    print(f'Best val Acc: {best_acc:4f} at epoch {best_epoch}')

    # load best model weights
    model.load_state_dict(best_model_wts)
    
    return model, training_curves

In [11]:
# Training
# loss and optimizer
criterion = nn.BCELoss() # BCELoss for binary classification
optimizer = torch.optim.Adam(mimic_admission_classifier.parameters(), lr=learning_rate)
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=0.95)

# Train the model. We also will store the results of training to visualize
mimic_admission_classifier, training_curves = train_model(mimic_admission_classifier, dataloaders, dataset_sizes, criterion, optimizer, scheduler, num_epochs=num_epochs)


Epoch 1/10
----------


C:\Users\Vinicius\AppData\Local\Temp\ipykernel_29824\1375628816.py:21: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  inputs = torch.tensor(self.mimic_df_X.iloc[idx], dtype=torch.float64)
C:\Users\Vinicius\AppData\Local\Temp\ipykernel_29824\1375628816.py:22: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  labels = torch.tensor(self.mimic_df_y.iloc[idx], dtype=torch.float64)


train Loss: 0.5140 Acc: 52.2381
val   Loss: 0.5050 Acc: 52.3356

Epoch 2/10
----------
